# Подготовка данных
Необходимо создать excel или использовать готовую таблицу с некоторыми тестовыми данными

In [1]:
import pandas as pd
import math
import time
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import List
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

#df = pd.read_excel("tests6.xlsx")
#df = pd.read_excel("tests_md.xlsx")
df = pd.read_excel("tests_lg.xlsx")
df.head()

,Id,Direction,Section,TestCaseName,Automated,Preconditions,Steps,Postconditions,ExpectedResult,TestData,Comments,Iterations,Priority,State,CreatedDate,CreatedById,Tags
0,556698.0,NaN,AI_Отладка,Общий шаг №1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medium,NotReady,06/24/2024 08:24:52,Временная техническая УЗ,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,Пример шага,NaN,Пример результата,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,Ntcn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Тест,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Highest,NotReady,05/23/2024 14:48:45,Служебный пользователь для Автотестов,"автоматизирован, ат_запуск, выдан, смоук хф, f..."


In [2]:
if torch.cuda.is_available():
    torch.cuda.get_device_properties('cuda')
else:
    pass

In [3]:
# Переключаем PyTorch на использование GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

# Начало обработки

Удаляем ненужные столбцы, заполняем нужные ячейки дублирующими значениями

In [4]:
start_timer = time.time()

def clean_dataset(dataset: pd.DataFrame) -> pd.DataFrame: 
    output_dataset = dataset[["Id", "Direction", "Section", "TestCaseName", "Preconditions", "Steps", "Postconditions", "ExpectedResult"]].copy()
    output_dataset[["Id", "Direction", "Section", "TestCaseName"]] = output_dataset[["Id", "Direction", "Section", "TestCaseName"]].ffill()
    return output_dataset

output = clean_dataset(df)
end_timer = time.time()

print(end_timer - start_timer)
output.to_excel("output.xlsx", index = False)
output.head()

0.0027017593383789062


,Id,Direction,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
0,556698.0,NaN,AI_Отладка,Общий шаг №1,NaN,NaN,NaN,NaN
1,556698.0,NaN,AI_Отладка,Общий шаг №1,NaN,Пример шага,NaN,Пример результата
2,556698.0,NaN,AI_Отладка,Общий шаг №1,NaN,Ntcn,NaN,NaN
3,556698.0,NaN,AI_Отладка,Общий шаг №1,NaN,Тест,NaN,NaN
4,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),NaN,NaN,NaN,NaN


# Разбиваем тест-кейсы на отдельные DataFrame

На выходе получим массив датафреймов(тест-кейсов)

In [5]:
start_timer = time.time()

def parse_tests_by_id(dataset: pd.DataFrame) -> List[pd.DataFrame]:
    test_cases = []
    for group_name, frame in dataset.groupby('Id'):
        test_cases.append(frame)

    return test_cases

output_array = parse_tests_by_id(output)
end_timer = time.time()
print(end_timer - start_timer)
output_array[0]

0.0026602745056152344


,Id,Direction,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
4,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),NaN,NaN,NaN,NaN
5,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),lol kek 2,NaN,NaN,cheburek 2
6,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),hehe,NaN,NaN,aboba
7,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),aboba,NaN,NaN,lolol))
8,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),Anton test,NaN,NaN,Anton good answer
9,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),LOL,NaN,NaN,ABOBA
10,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),Открыта страница входа в ЕФР,NaN,NaN,NaN
11,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),NaN,"В поле ""Домен"" из выпадающего списка выбрать ""GO""",NaN,Выбранное значение сохраняется в поле
12,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),NaN,"В поле ""Логин"" ввести логин пользователя",NaN,Введенное значение сохраняется в поле
13,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),NaN,"В поле ""АБОБА"" ввести пароль пользователя",NaN,Введенное значение сохраняется в поле


In [6]:
output_array[1]

,Id,Direction,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
15,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,NaN,NaN,NaN,NaN
16,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,lol kek 2,NaN,NaN,cheburek 2
17,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,aboba,NaN,NaN,lolol))
18,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,Anton test,NaN,NaN,Anton good answer
19,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,LOL,NaN,NaN,ABOBA
20,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,Открыть страницу входа в ЕФР,NaN,NaN,"Внизу экрана, под кнопкой ""Войти"", расположен ..."
21,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,Нажать кнопку 8888@rshb.intech.ru,NaN,NaN,Открывается Лотус на странице создания письма
22,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,"Проверить поле ""Кому""",NaN,NaN,"В поле ""Кому"" указано 8888@rshb.intech.ru"
23,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,тестовый шаг 3,NaN,NaN,тестовый ответ 3


In [7]:
#output_array[2]

# Удаление пустых ячеек

In [8]:
start_timer = time.time()

def remove_empty_cells(array):
    # обрабатываем "Preconditions", "Steps", "Postconditions", "ExpectedResult" (удаляем первую ячейку)
    def up_cells(tst_case, column):
        for i in range(len(tst_case[column])):
            if i == len(tst_case.index) - 1:
                tst_case.at[tst_case.index[i], column] = math.nan
                break
            else:
                tst_case.at[tst_case.index[i], column] = tst_case.at[tst_case.index[i + 1], column]
                
        return tst_case
        
    for dframe in array:
        for col in ["Preconditions", "Steps", "Postconditions", "ExpectedResult"]:
            dframe = up_cells(dframe, col)
              
        dframe["Steps"] = dframe["Steps"].fillna(dframe["Preconditions"])
        dframe["Steps"] = dframe["Steps"].fillna(dframe["Postconditions"])
        dframe.drop(["Preconditions", "Postconditions"], inplace=True, axis=1)     
            
    return array

edited_output_array = remove_empty_cells(output_array)
end_timer = time.time()
print(end_timer - start_timer)
edited_output_array[0]

0.23570537567138672


,Id,Direction,Section,TestCaseName,Steps,ExpectedResult
4,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),lol kek 2,cheburek 2
5,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),hehe,aboba
6,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),aboba,lolol))
7,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),Anton test,Anton good answer
8,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),LOL,ABOBA
9,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),Открыта страница входа в ЕФР,NaN
10,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"В поле ""Домен"" из выпадающего списка выбрать ""GO""",Выбранное значение сохраняется в поле
11,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"В поле ""Логин"" ввести логин пользователя",Введенное значение сохраняется в поле
12,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"В поле ""АБОБА"" ввести пароль пользователя",Введенное значение сохраняется в поле
13,544009.0,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"Нажать кнопку ""Войти""","Авторизация успешна. Открылась страница ""Поиск..."


In [9]:
pd.DataFrame(pd.concat(edited_output_array)).to_excel("edited_output.xlsx", index=False)

In [10]:
edited_output_array[1]

,Id,Direction,Section,TestCaseName,Steps,ExpectedResult
15,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,lol kek 2,cheburek 2
16,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,aboba,lolol))
17,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,Anton test,Anton good answer
18,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,LOL,ABOBA
19,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,Открыть страницу входа в ЕФР,"Внизу экрана, под кнопкой ""Войти"", расположен ..."
20,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,Нажать кнопку 8888@rshb.intech.ru,Открывается Лотус на странице создания письма
21,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,"Проверить поле ""Кому""","В поле ""Кому"" указано 8888@rshb.intech.ru"
22,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,тестовый шаг 3,тестовый ответ 3
23,544010.0,AI_Отладка,1.1. Аутентификация пользователя,Кнопка 8888@rshb.intech.ru для отправки письма...,NaN,NaN


# (Опционально) Замена тест-кейсов в массиве на наборы текстовых предложений

In [11]:
# Предложения для теста

# 0. Во время обучения модели machine learning, важно обеспечить достаточное количество данных для тренировки.
# 1. Во время обучения модели машинного обучения, важно обеспечить достаточное количество данных для тренировки.

# 2. Использование API позволяет интегрировать различные сервисы в ваше приложение, упрощая процесс разработки.
# 3. The use of an API allows you to integrate various services into your application, simplifying the development process.

# 4. Для анализа больших объемов данных рекомендуется использовать фреймворк Hadoop, который обеспечивает эффективную обработку данных.
# 5. В процессе дебаггинга программы была обнаружена ошибка, связанная с неправильным использованием переменных.

# 6. При разработке user interface необходимо учитывать принципы user experience, чтобы сделать приложение максимально удобным для пользователя.
# 7. При разработке пользовательского интерфейса необходимо учитывать принципы пользовательского опыта, чтобы сделать приложение максимально удобным для пользователя.

# 8. Летний отпуск всегда запоминается яркими моментами, проведёнными на берегу моря.
# 9. Летний отпуск всегда оставляет яркие впечатления, проведённые на пляже.

# 10. Вчера мы с друзьями проводили время на природе, наслаждаясь пикником и свежим воздухом.
# 11. Вчера мы с друзьями провели время в помещении, скучая и наслаждаясь теплом.

# 12. The latest trends in fashion emphasize sustainability and eco-friendly materials.
# 13. The latest trends in fashion ignore sustainability and promote fast fashion.

# 14. В этом году зима выдалась особенно холодной, и снег покрыл все окрестности белым покрывалом.

def switch_to_lang(array):
    array.clear()
    array.append("Во время обучения модели machine learning, важно обеспечить достаточное количество данных для тренировки") 
    array.append("Во время обучения модели машинного обучения, важно обеспечить достаточное количество данных для тренировки")
    
    array.append("Использование API позволяет интегрировать различные сервисы в ваше приложение, упрощая процесс разработки") 
    array.append("The use of an API allows you to integrate various services into your application, simplifying the development process")
    
    array.append("Для анализа больших объемов данных рекомендуется использовать фреймворк Hadoop, который обеспечивает эффективную обработку данных") 
    array.append("В процессе дебаггинга программы была обнаружена ошибка, связанная с неправильным использованием переменных")
    
    array.append("При разработке user interface необходимо учитывать принципы user experience, чтобы сделать приложение максимально удобным для пользователя") 
    array.append("При разработке пользовательского интерфейса необходимо учитывать принципы пользовательского опыта, чтобы сделать приложение максимально удобным для пользователя")
    
    array.append("Летний отпуск всегда запоминается яркими моментами, проведёнными на берегу моря")
    array.append("Летний отпуск всегда оставляет яркие впечатления, проведённые на пляже")
    
    array.append("Вчера мы с друзьями проводили время на природе, наслаждаясь пикником и свежим воздухом.")
    array.append("Вчера мы с друзьями провели время в помещении, скучая и наслаждаясь теплом")
    
    array.append("The latest trends in fashion emphasize sustainability and eco-friendly materials")
    array.append("The latest trends in fashion ignore sustainability and promote fast fashion")
    
    array.append("В этом году зима выдалась особенно холодной, и снег покрыл все окрестности белым покрывалом")
    return array

#edited_output_array = switch_to_lang(edited_output_array)

len(edited_output_array)

247

# (Опционально) Убираем поле id

In [12]:
def remove_ids(dataset):
    array = []
    for dframe in dataset:
        array.append(dframe[["Direction", "Section", "TestCaseName", "Steps", "ExpectedResult"]])
        
    return array

edited_output_array = remove_ids(edited_output_array)
edited_output_array[0]

,Direction,Section,TestCaseName,Steps,ExpectedResult
4,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),lol kek 2,cheburek 2
5,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),hehe,aboba
6,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),aboba,lolol))
7,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),Anton test,Anton good answer
8,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),LOL,ABOBA
9,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),Открыта страница входа в ЕФР,NaN
10,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"В поле ""Домен"" из выпадающего списка выбрать ""GO""",Выбранное значение сохраняется в поле
11,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"В поле ""Логин"" ввести логин пользователя",Введенное значение сохраняется в поле
12,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"В поле ""АБОБА"" ввести пароль пользователя",Введенное значение сохраняется в поле
13,AI_Отладка,1.1. Аутентификация пользователя,Вход в систему ЕФР (GO),"Нажать кнопку ""Войти""","Авторизация успешна. Открылась страница ""Поиск..."


# spaCy

```conda install spacy```

```python -m spacy download ru_core_news_lg```


In [13]:
import spacy

# Переключаем spaCy на использование GPU, если доступно
# spacy.prefer_gpu()
# spacy.require_gpu()

start_timer = time.time()

spacy_nlp = spacy.load("ru_core_news_lg")

def vectorize_text(text: str, spacy_model: spacy.Language) -> torch.Tensor:
    doc = spacy_model(text)
    return torch.from_numpy(doc.vector.reshape(1, -1))
    # if type(doc.vector) is cp.ndarray:
    #     return torch.from_numpy(cp.asnumpy(doc.vector).reshape(1, -1))
    # else:
    #     return torch.from_numpy(doc.vector.reshape(1, -1))

spacy_vectorized_output_array = []

for test_case in edited_output_array:
    spacy_vectorized_output_array.append(F.normalize(vectorize_text(str(test_case), spacy_nlp), p=2, dim=1))

def output_spacy_to_csv():
    for i in range(len(spacy_vectorized_output_array)):
        pd.DataFrame(spacy_vectorized_output_array[i]).to_csv("spacy_output/spacy_vectorized_output_" + str(i) + ".csv", index = False)    

output_spacy_to_csv()

end_timer = time.time()
print(end_timer - start_timer)
spacy_vectorized_output_array[0]

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class 'pandas.core.frame.DataFrame'>

# sentence-transformers/paraphrase-multilingual-mpnet-base-v2

```conda install -c conda-forge sentence-transformers```

https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2

In [ ]:
from transformers import AutoTokenizer, AutoModel

start_timer = time.time()

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
mpnet_vectorized_output_array = []

model.to(device)

for test_case in edited_output_array:
    # Tokenize
    encoded_input = tokenizer(str(test_case), return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, average pooling
    embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    mpnet_vectorized_output_array.append(F.normalize(embedding, p=2, dim=1))

def output_mpnet_to_csv():
    for i in range(len(mpnet_vectorized_output_array)):
        pd.DataFrame(mpnet_vectorized_output_array[i].cpu()).to_csv("mpnet_output/mpnet_vectorized_output_" + str(i) + ".csv", index = False)

output_mpnet_to_csv()

end_timer = time.time()
print(end_timer - start_timer)
mpnet_vectorized_output_array

# google-bert/bert-base-multilingual-cased 

https://huggingface.co/google-bert/bert-base-multilingual-cased

In [ ]:
from transformers import BertTokenizer, BertModel

start_timer = time.time()

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained('bert-base-multilingual-cased')
bert_vectorized_output_array = []

bert_model.to(device)

def bert_vectorize_text(text: str) -> torch.Tensor:
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)

    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return F.normalize(cls_embedding, p=2, dim=1)

for test_case in edited_output_array:
    bert_vectorized_output_array.append(bert_vectorize_text(str(test_case)))

def output_bert_to_csv() -> None:
    for i in range(len(bert_vectorized_output_array)):
        pd.DataFrame(bert_vectorized_output_array[i].cpu()).to_csv("bert_output/bert_vectorized_output_" + str(i) + ".csv", index = False)

output_bert_to_csv()

end_timer = time.time()
print(end_timer - start_timer)
bert_vectorized_output_array

# intfloat/multilingual-e5-large-instruct

https://huggingface.co/intfloat/multilingual-e5-large-instruct

In [ ]:
# from sentence_transformers import SentenceTransformer
# 
# start_timer = time.time()
# 
# model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')
from transformers import AutoTokenizer, AutoModel

start_timer = time.time()

def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large-instruct')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large-instruct')
e5_vectorized_output_array = []

model.to(device)

for test_case in edited_output_array:
    # Tokenize the input texts
    batch_dict = tokenizer(str(test_case), padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**batch_dict)

    emb = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
    e5_vectorized_output_array.append(F.normalize(emb, p=2, dim=1))


def output_e5_to_csv():
    for i in range(len(e5_vectorized_output_array)):
        pd.DataFrame(e5_vectorized_output_array[i].cpu()).to_csv("e5_output/e5_vectorized_output_" + str(i) + ".csv", index = False)

output_e5_to_csv()

end_timer = time.time()
print(end_timer - start_timer)
e5_vectorized_output_array

# ai-forever/sbert_large_nlu_ru

https://huggingface.co/ai-forever/sbert_large_nlu_ru

In [ ]:
from transformers import AutoTokenizer, AutoModel

start_timer = time.time()

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask) -> torch.Tensor:
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
sbert_vectorized_output_array = []

model.to(device)

for test_case in edited_output_array:
    encoded_input = tokenizer(str(test_case), padding=True, truncation=True, return_tensors='pt', max_length=512).to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    emb = mean_pooling(model_output, encoded_input['attention_mask'])
    sbert_vectorized_output_array.append(F.normalize(emb, p=2, dim=1))

def output_sbert_to_csv():
    for i in range(len(sbert_vectorized_output_array)):
        pd.DataFrame(sbert_vectorized_output_array[i].cpu()).to_csv("sbert_output/sbert_vectorized_output_" + str(i) + ".csv", index = False)

output_sbert_to_csv()
    
end_timer = time.time()
print(end_timer - start_timer)
sbert_vectorized_output_array

In [ ]:
print((spacy_vectorized_output_array[0][0]).shape)
print((mpnet_vectorized_output_array[0][0]).shape)
print((bert_vectorized_output_array[0][0]).shape)
print((e5_vectorized_output_array[0][0]).shape)
print((sbert_vectorized_output_array[0][0]).shape)

# Оценка результатов (Косинусное сходство)

Для оценки семантики векторизованных тест-кейсов будет использоваться косинусное сходство.
Можно обратить внимание на блок ниже: нет никакого смысла считать косинусное сходство одинаковых предложений, обработанных разными эмбединговыми моделями.

In [ ]:
def count_cosine_similarity(array1: List[torch.Tensor], array2: List[torch.Tensor]):
    result = []
    for i, j in zip(array1, array2):
         result.append(F.cosine_similarity(i, j))

    return result

mpnet_bert = count_cosine_similarity(mpnet_vectorized_output_array, bert_vectorized_output_array)
e5_sbert = count_cosine_similarity(e5_vectorized_output_array, sbert_vectorized_output_array)

print(mpnet_bert)
print(e5_sbert)

Поэтому, для начала необходимо посчитать косинусное сходство между разными предложениями внутри одной модели (данный блок актуален только для 8ми предложений)

In [ ]:
print(len(spacy_vectorized_output_array[0]))
print(len(spacy_vectorized_output_array[0][0]))
print(type(spacy_vectorized_output_array[0]))
print(len(spacy_vectorized_output_array[1]))
print(len(spacy_vectorized_output_array[1][0]))

In [ ]:
# 0. Во время обучения модели machine learning, важно обеспечить достаточное количество данных для тренировки.
# 1. Во время обучения модели машинного обучения, важно обеспечить достаточное количество данных для тренировки.

# 2. Использование API позволяет интегрировать различные сервисы в ваше приложение, упрощая процесс разработки.
# 3. The use of an API allows you to integrate various services into your application, simplifying the development process.

# 4. Для анализа больших объемов данных рекомендуется использовать фреймворк Hadoop, который обеспечивает эффективную обработку данных.
# 5. В процессе дебаггинга программы была обнаружена ошибка, связанная с неправильным использованием переменных.

# 6. При разработке user interface необходимо учитывать принципы user experience, чтобы сделать приложение максимально удобным для пользователя.
# 7. При разработке пользовательского интерфейса необходимо учитывать принципы пользовательского опыта, чтобы сделать приложение максимально удобным для пользователя.

# 8. Летний отпуск всегда запоминается яркими моментами, проведёнными на берегу моря.
# 9. Летний отпуск всегда оставляет яркие впечатления, проведённые на пляже.

# 10. Вчера мы с друзьями проводили время на природе, наслаждаясь пикником и свежим воздухом.
# 11. Вчера мы с друзьями провели время в помещении, скучая и наслаждаясь теплом.

# 12. The latest trends in fashion emphasize sustainability and eco-friendly materials.
# 13. The latest trends in fashion ignore sustainability and promote fast fashion.

# 14. В этом году зима выдалась особенно холодной, и снег покрыл все окрестности белым покрывалом.

def sentence_cosine_similarity(array):
    first_case = F.cosine_similarity(array[0], array[1]) # два слова заменены с английского на русский
    second_case = F.cosine_similarity(array[2], array[3]) # два одинаковых предложения на русском и английском
    third_case = F.cosine_similarity(array[4], array[5]) # два разных предложения, но находятся в общем контексте
    fourth_case = F.cosine_similarity(array[6], array[7]) # два словосочетания заменены с английского на русский
    fifth_case = F.cosine_similarity(array[0], array[7]) # два разных предложения, но находятся они в одном общем контексте
    sixth_case = F.cosine_similarity(array[8], array[9]) # два синонимичных предложения
    seventh_case = F.cosine_similarity(array[10], array[11]) # два антонима
    eighth_case = F.cosine_similarity(array[12], array[13]) # два антонима на английском
    ninth_case = F.cosine_similarity(array[14], array[1]) # два разных предложения с разным контекстом
    
    return [first_case, second_case, third_case, fourth_case, fifth_case, sixth_case, seventh_case, eighth_case, ninth_case]

spacy_cosine_similarity = sentence_cosine_similarity(spacy_vectorized_output_array)
mpnet_cosine_similarity = sentence_cosine_similarity(mpnet_vectorized_output_array)
bert_cosine_similarity = sentence_cosine_similarity(bert_vectorized_output_array)
e5_cosine_similarity = sentence_cosine_similarity(e5_vectorized_output_array)
sbert_cosine_similarity = sentence_cosine_similarity(sbert_vectorized_output_array)

print("spacy", spacy_cosine_similarity)
print("mpnet", mpnet_cosine_similarity)
print("bert", bert_cosine_similarity)
print("e5", e5_cosine_similarity)
print("sbert", sbert_cosine_similarity)
# mpnet_vectorized_output_array[2] @ mpnet_vectorized_output_array[3].T

# Оценка результатов (Евклидово расстояние)

In [ ]:
def sentence_euclidean_distance(array):
    count_euclidean_d = lambda tensor_a, tensor_b: torch.sqrt(torch.sum(torch.pow((tensor_a - tensor_b), 2)))
    
    first_case = count_euclidean_d(array[0], array[1]) # два слова заменены с английского на русский
    second_case = count_euclidean_d(array[2], array[3]) # два одинаковых предложения на русском и английском
    third_case = count_euclidean_d(array[4], array[5]) # два разных предложения, но находятся в общем контексте
    fourth_case = count_euclidean_d(array[6], array[7]) # два словосочетания заменены с английского на русский
    fifth_case = count_euclidean_d(array[0], array[7]) # два разных предложения, но находятся они в одном общем контексте
    sixth_case = count_euclidean_d(array[8], array[9]) # два синонимичных предложения
    seventh_case = count_euclidean_d(array[10], array[11]) # два антонима
    eighth_case = count_euclidean_d(array[12], array[13]) # два антонима на английском
    ninth_case = count_euclidean_d(array[14], array[1]) # два разных предложения с разным контекстом
    
    return [first_case, second_case, third_case, fourth_case, fifth_case, sixth_case, seventh_case, eighth_case, ninth_case]

spacy_euclidean_distance = sentence_euclidean_distance(spacy_vectorized_output_array)
mpnet_euclidean_distance = sentence_euclidean_distance(mpnet_vectorized_output_array)
bert_euclidean_distance = sentence_euclidean_distance(bert_vectorized_output_array)
e5_euclidean_distance = sentence_euclidean_distance(e5_vectorized_output_array)
sbert_euclidean_distance = sentence_euclidean_distance(sbert_vectorized_output_array)

print("spacy", spacy_euclidean_distance)
print("mpnet", mpnet_euclidean_distance)
print("bert", bert_euclidean_distance)
print("e5", e5_euclidean_distance)
print("sbert", sbert_euclidean_distance)

# Оценка результатов (t-SNE)

In [ ]:
def tsne_emb(array):
    embeddings = []
    for t_outer in array:
        for t_inner in t_outer:
            embeddings.append(t_inner.cpu().numpy())
        
    # k = 4  # Количество кластеров
    # kmeans = KMeans(n_clusters=k, random_state=42)
    # clusters = kmeans.fit_predict(embeddings)

    # Визуализация с использованием t-SNE
    perplexity_value = min(30, len(embeddings) - 1)  # Не больше, чем количество образцов
    tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_value)
    embeddings_2d = tsne.fit_transform(np.array(embeddings))
    
    return embeddings_2d

tsne_emb(mpnet_vectorized_output_array)

# Перевод данных из тензоров в векторы

In [ ]:
def clean_array_from_tensors(array):
    cleaned_array = []
    for t in array:
        cleaned_array.append(t.cpu().numpy())

    return cleaned_array

spacy_cosine_similarity_cleaned = clean_array_from_tensors(spacy_cosine_similarity)
mpnet_cosine_similarity_cleaned = clean_array_from_tensors(mpnet_cosine_similarity)
bert_cosine_similarity_cleaned = clean_array_from_tensors(bert_cosine_similarity)
e5_cosine_similarity_cleaned = clean_array_from_tensors(e5_cosine_similarity)
sbert_cosine_similarity_cleaned = clean_array_from_tensors(sbert_cosine_similarity)

spacy_euclidean_distance_cleaned = clean_array_from_tensors(spacy_euclidean_distance)
mpnet_euclidean_distance_cleaned = clean_array_from_tensors(mpnet_euclidean_distance)
bert_euclidean_distance_cleaned = clean_array_from_tensors(bert_euclidean_distance)
e5_euclidean_distance_cleaned = clean_array_from_tensors(e5_euclidean_distance)
sbert_euclidean_distance_cleaned = clean_array_from_tensors(sbert_euclidean_distance)

print(spacy_cosine_similarity_cleaned)
print(spacy_euclidean_distance_cleaned)

# Визуализация результатов

# Косинунсное сходство

In [ ]:
heatmap_cosine_similarity = np.empty((5, 9))
# for j in range(len(heatmap_cosine_similarity)):
#     heatmap_cosine_similarity[0][j] = spacy_cosine_similarity_cleaned[j][0]
#     heatmap_cosine_similarity[1][j] = mpnet_cosine_similarity_cleaned[j][0]
#     heatmap_cosine_similarity[2][j] = bert_cosine_similarity_cleaned[j][0]
#     heatmap_cosine_similarity[3][j] = e5_cosine_similarity_cleaned[j][0]
#     heatmap_cosine_similarity[4][j] = sbert_cosine_similarity_cleaned[j][0]

for i in range(5):
    for j in range(9):
        heatmap_cosine_similarity[0][j] = spacy_cosine_similarity_cleaned[j]
        heatmap_cosine_similarity[1][j] = mpnet_cosine_similarity_cleaned[j]
        heatmap_cosine_similarity[2][j] = bert_cosine_similarity_cleaned[j]
        heatmap_cosine_similarity[3][j] = e5_cosine_similarity_cleaned[j]
        heatmap_cosine_similarity[4][j] = sbert_cosine_similarity_cleaned[j]

print(heatmap_cosine_similarity)

plt.figure(figsize=(16, 12))
ax = sns.heatmap(heatmap_cosine_similarity, annot=True, cmap='inferno', cbar_kws={'label': 'Cosine Similarity'},
                 yticklabels=['SpaCy', 'mpnet v2', 'bert', 'e5', 'sbert'],
                 xticklabels=["First case", "Second case", "Third case", "Fourth case", "Fifth case", "Sixth_case", "Seventh_case", "Eighth_case", "Ninth_case"], fmt='.3f')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
plt.title("Cosine Similarity Heatmap")
plt.xlabel("Cases")
plt.ylabel("Model names")
plt.show()

# Евклидово расстояние

In [ ]:
heatmap_euclidean_distance = np.empty((5, 9))
for i in range(5):
    for j in range(9):
        heatmap_euclidean_distance[0][j] = spacy_euclidean_distance_cleaned[j]
        heatmap_euclidean_distance[1][j] = mpnet_euclidean_distance_cleaned[j]
        heatmap_euclidean_distance[2][j] = bert_euclidean_distance_cleaned[j]
        heatmap_euclidean_distance[3][j] = e5_euclidean_distance_cleaned[j]
        heatmap_euclidean_distance[4][j] = sbert_euclidean_distance_cleaned[j]
     
print(heatmap_euclidean_distance)

plt.figure(figsize=(16, 12))
ax = sns.heatmap(heatmap_euclidean_distance, annot=True, cmap='inferno', cbar_kws={'label': 'Euclidean Distance'},
                 yticklabels=['SpaCy', 'mpnet v2', 'bert', 'e5', 'sbert'],
                 xticklabels=["First case", "Second case", "Third case", "Fourth case", "Fifth case", "Sixth_case", "Seventh_case", "Eighth_case", "Ninth_case"], fmt='.3f')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)
plt.title("Euclidean Distance Heatmap")
plt.xlabel("Cases")
plt.ylabel("Model names")
plt.show()

# t-SNE

In [ ]:
data_sets = [spacy_vectorized_output_array, mpnet_vectorized_output_array, bert_vectorized_output_array, e5_vectorized_output_array, sbert_vectorized_output_array]
labels = ['Spacy', 'mpnet v2', 'bert', 'e5', 'sbert']  # Подписи для легенды
colors = ['r', 'g', 'b', 'c', 'm']  # Цвета для разных наборов

plt.figure(figsize=(20, 15))

for data, label, color in zip(data_sets, labels, colors):
    # Получение проекций
    embeddings_2d = tsne_emb(data)

    # Наносим точки на график
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1],
                label=label, color=color, alpha=0.5, s=100)

    #for i in range(len(embeddings_2d)):
       # plt.text(embeddings_2d[i, 0], embeddings_2d[i, 1], str(i+1), fontsize=9, ha='center', va='center', color='white')

plt.title('t-SNE Visualization of Multiple Datasets')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.legend()
plt.show()

# K means (Spacy)

In [ ]:
spacy_cluster_array = []
for t_outer in spacy_vectorized_output_array:
        for t_inner in t_outer:
            spacy_cluster_array.append(t_inner.cpu().numpy())

spacy_cluster_array = np.array(spacy_cluster_array)
# Настройка K-means
n_clusters = 3  # Укажите количество кластеров
spacy_kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Обучение K-means
spacy_kmeans.fit(spacy_cluster_array)

# Получите метки кластеров и центры
spacy_labels = spacy_kmeans.labels_
spacy_centers = spacy_kmeans.cluster_centers_

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(spacy_cluster_array[:, 0], spacy_cluster_array[:, 1], c=spacy_labels, cmap='viridis', marker='o', edgecolor='k', s=50)
plt.scatter(spacy_centers[:, 0], spacy_centers[:, 1], c='red', marker='X', s=200, label='Центры кластеров')
plt.title('Результаты кластеризации с помощью K-means')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.legend()
plt.grid()
plt.show()

# K means (mpnet)

In [ ]:
mpnet_cluster_array = []
for t_outer in mpnet_vectorized_output_array:
        for t_inner in t_outer:
            mpnet_cluster_array.append(t_inner.cpu().numpy())

mpnet_cluster_array = np.array(mpnet_cluster_array)
# Настройка K-means
n_clusters = 3  # Укажите количество кластеров
mpnet_kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Обучение K-means
mpnet_kmeans.fit(mpnet_cluster_array)

# Получите метки кластеров и центры
mpnet_labels = mpnet_kmeans.labels_
mpnet_centers = mpnet_kmeans.cluster_centers_

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(mpnet_cluster_array[:, 0], mpnet_cluster_array[:, 1], c=mpnet_labels, cmap='viridis', marker='o', edgecolor='k', s=50)
plt.scatter(mpnet_centers[:, 0], mpnet_centers[:, 1], c='red', marker='X', s=200, label='Центры кластеров')
plt.title('Результаты кластеризации с помощью K-means')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.legend()
plt.grid()
plt.show()

# K means (bert)

In [ ]:
bert_cluster_array = []
for t_outer in bert_vectorized_output_array:
        for t_inner in t_outer:
            bert_cluster_array.append(t_inner.cpu().numpy())

bert_cluster_array = np.array(bert_cluster_array)
# Настройка K-means
n_clusters = 3  # Укажите количество кластеров
bert_kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Обучение K-means
bert_kmeans.fit(bert_cluster_array)

# Получите метки кластеров и центры
bert_labels = bert_kmeans.labels_
bert_centers = bert_kmeans.cluster_centers_

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(bert_cluster_array[:, 0], bert_cluster_array[:, 1], c=bert_labels, cmap='viridis', marker='o', edgecolor='k', s=50)
plt.scatter(bert_centers[:, 0], bert_centers[:, 1], c='red', marker='X', s=200, label='Центры кластеров')
plt.title('Результаты кластеризации с помощью K-means')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.legend()
plt.grid()
plt.show()

# K means (e5)

In [ ]:
e5_cluster_array = []
for t_outer in e5_vectorized_output_array:
        for t_inner in t_outer:
            e5_cluster_array.append(t_inner.cpu().numpy())

e5_cluster_array = np.array(e5_cluster_array)
# Настройка K-means
n_clusters = 3  # Укажите количество кластеров
e5_kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Обучение K-means
e5_kmeans.fit(e5_cluster_array)

# Получите метки кластеров и центры
e5_labels = e5_kmeans.labels_
e5_centers = e5_kmeans.cluster_centers_

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(e5_cluster_array[:, 0], e5_cluster_array[:, 1], c=e5_labels, cmap='viridis', marker='o', edgecolor='k', s=50)
plt.scatter(e5_centers[:, 0], e5_centers[:, 1], c='red', marker='X', s=200, label='Центры кластеров')
plt.title('Результаты кластеризации с помощью K-means')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.legend()
plt.grid()
plt.show()

# K means (sbert)

In [ ]:
sbert_cluster_array = []
for t_outer in sbert_vectorized_output_array:
        for t_inner in t_outer:
            sbert_cluster_array.append(t_inner.cpu().numpy())

sbert_cluster_array = np.array(sbert_cluster_array)
# Настройка K-means
n_clusters = 3  # Укажите количество кластеров
sbert_kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Обучение K-means
sbert_kmeans.fit(sbert_cluster_array)

# Получите метки кластеров и центры
sbert_labels = sbert_kmeans.labels_
sbert_centers = sbert_kmeans.cluster_centers_

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(sbert_cluster_array[:, 0], sbert_cluster_array[:, 1], c=sbert_labels, cmap='viridis', marker='o', edgecolor='k', s=50)
plt.scatter(sbert_centers[:, 0], sbert_centers[:, 1], c='red', marker='X', s=200, label='Центры кластеров')
plt.title('Результаты кластеризации с помощью K-means')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.legend()
plt.grid()
plt.show()